# Workshop Homework

## Question 1. dlt Version

### Installing dlt with Qdrant

In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [3]:
import dlt

print(dlt.__version__)

1.12.3


### dlt Resource

In [5]:
import requests


@dlt.resource(name="zoomcamp_data", write_disposition='replace')
def zoomcamp_data():
    docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course["course"]

        for doc in course["documents"]:
            doc["course"] = course_name
            yield doc

## Question 2. dlt pipeline

In [6]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(qd_path='db.qdrant')

In [8]:
pipeline = dlt.pipeline(pipeline_name="zoomcamp_pipeline", destination=qdrant_destination, dataset_name="zoomcamp_tagged_data")
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Run started at 2025-07-06 12:27:28.856654+00:00 and COMPLETED in 15.75 seconds with 4 steps.
Step extract COMPLETED in 1.18 seconds.

Load package 1751804854.4238427 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.06 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1751804854.4238427 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 8.94 seconds.
Pipeline zoomcamp_pipeline load step completed in 8.93 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/wgb/llm-zoomcamp-hw/db.qdrant location to store data
Load package 1751804854.4238427 is LOADED and contains no failed jobs

Step run COMPLETED in 15.75 seconds.
Pipeline zoomcamp_pipeline load step completed in 8.93 seconds
1 load package(s) were loaded to destination qdr

948 rows were inserted into the zoomcamp_data collection

### Question 3. Embeddings

Since the meta.json file mentions `fast-bge-small-en`, the `BAAI/bge-small-en` embedding model was used.